In [1]:
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(font_scale=1.5, palette='Set2')
sns.set_style('whitegrid')

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '..')
import dsdna_mpra
from dsdna_mpra import config, plots

GenBank feature annotations are downloaded using the [`download_genbank_features.py`](../scripts/download_genbank_features.py) script.

Overlaps between genomic features, tiles, and CREs are computed using the [`cre_genomic_feature_overlap.py`](../scripts/cre_genomic_feature_overlap.py) script.


**Note**: The 'expected' overlap is calculated under a null model in which CRE positions are assumed to be uniformly randomly distributed across the genome.


### Overlap with Coding Regions

Tile activity stratified by coding vs. non-coding regions, as annotated in GenBank records


In [2]:
paired_tiles_cds = pd.read_csv(config.RESULTS_DIR / "virus_paired_tiles_cds_overlap.csv")
paired_tiles_cds['virus'] = (paired_tiles_cds.family + ', ' + paired_tiles_cds.strain).astype('category').cat.set_categories(config.VIRUSES)
paired_tiles_cds.family = paired_tiles_cds.family.astype('category').cat.set_categories(config.DSDNA_FAMILIES[::-1])
paired_tiles_cds.cell = paired_tiles_cds.cell.astype('category').cat.set_categories(config.CELL_LINES)
paired_tiles_cds.sample(1, random_state=0)

,family,strain,cell,genome,begin,fwd_lfc,rev_lfc,tile_id,threshold,tile_lfc,end,cell_rank,is_cds,virus
211684,Papillomaviridae,Type 11,MRC5,M14119.1,2600,1.078173,0.930253,Papilloma_Virus:Type_11:53:+;Papilloma_Virus:T...,1.800548,1.078173,2800,0.279405,True,"Papillomaviridae, Type 11"


Heatmap: median cell activity rank of CDS vs. non-CDS tiles

In [3]:
grouped = (
    paired_tiles_cds
    .groupby(['family', 'virus', 'cell', 'is_cds'], observed=True)['cell_rank']
    .median()
    .reset_index()
)
grouped['cds_label'] = grouped['is_cds'].map({False: 'not CDS', True: 'CDS'})
grouped['cell_is_cds'] = grouped['cell'].astype(str) + ', ' + grouped['cds_label']
sorted_cells = list(paired_tiles_cds['cell'].cat.categories)

column_order = []
for cell in sorted_cells:
    column_order.append(f"{cell}, not CDS")
    column_order.append(f"{cell}, CDS")

heatmap_data = grouped.pivot_table(
    index=['family', 'virus'],
    columns='cell_is_cds',
    values='cell_rank',
    observed=False
)
heatmap_data = heatmap_data.sort_index(level=['family', 'virus'])
heatmap_data.index = heatmap_data.index.droplevel('family')
existing_cols = [col for col in column_order if col in heatmap_data.columns]
heatmap_data = heatmap_data[existing_cols]

plt.figure(figsize=(20, 15))
ax = sns.heatmap(heatmap_data, cmap='viridis', linewidths=0.5, linecolor='gray')
ax.set_yticks(np.arange(len(heatmap_data)) + 0.5)
ax.set_yticklabels(heatmap_data.index.tolist(), rotation=0)
colorbar = ax.collections[0].colorbar
colorbar.set_label("median cell activity rank")
ax.set_xlabel("")
ax.set_ylabel("")
ax.grid(False)
plt.savefig(config.FIGURES_DIR / "tiles_cds_overlap_median_rank_activity.pdf", format="pdf", bbox_inches="tight")
plt.close()

Distribution of cell activity rank of CDS vs. non-CDS tiles

In [4]:
CELL = 'K562'
cell_tiles = paired_tiles_cds[paired_tiles_cds.cell == CELL].copy()

fig, ax = plt.subplots(figsize=(35, 15), layout="tight")
n_viruses = len(config.VIRUSES)
X_STEP = 6
CDS_OFFSET = 2  # space between not-CDS and CDS
major_xticks = []
minor_xticks = []
minor_labels = []
for virus_idx, (virus, virus_tb) in enumerate(cell_tiles.groupby('virus', observed=True)):
    base_x = virus_idx * X_STEP
    not_cds_x = base_x
    cds_x = base_x + CDS_OFFSET
    not_cds = virus_tb[~virus_tb.is_cds].tile_lfc.values
    cds = virus_tb[virus_tb.is_cds].tile_lfc.values
    plots.violin(ax, not_cds, not_cds_x, width_factor=1, box_width=.1)
    plots.violin(ax, cds, cds_x, width_factor=1, box_width=.1)
    major_xticks.append((not_cds_x + cds_x) / 2)
    minor_xticks.extend([not_cds_x, cds_x])
    minor_labels.extend(['not CDS', 'CDS'])
ax.set_ylim([0, 8])
ax.set_ylabel(r'$\log_2 (FC + 1)$')
ax.tick_params(which='major', pad=20)
ax.set_xticks(major_xticks)
ax.set_xticklabels(config.VIRUSES, rotation=90)
ax.set_xticks(minor_xticks, minor=True)
ax.set_xticklabels(minor_labels, minor=True, rotation=90, fontsize=15)
plt.savefig(config.FIGURES_DIR / "tiles_cds_overlap_rank_activity_distribution.pdf", format="pdf", bbox_inches="tight")
plt.close()


Fraction of viral CREs in CDSs: observed vs. expected fractions

In [5]:
cres_cds_fractions_df = pd.read_csv(config.RESULTS_DIR / "cres_in_cds_fractions.csv")

fig, ax = plt.subplots(figsize=(10, 10), layout="tight")
ax.set_title('Fraction of viral CREs in CDSs', fontsize=30)
for family, fraction_df in cres_cds_fractions_df.groupby('family', sort=True, observed=True):
    ax.scatter(fraction_df.expected, fraction_df.observed, alpha=.7, s=250, color=config.DSDNA_FAMILY_COLORS[family], label=family)  # add outline: edgecolor='black'
ax.plot([0, 1], [0, 1], ls='-', color='black')
ax.legend(fontsize=20, loc='upper left')
ax.set_xlabel('Expected', fontsize=25)
ax.set_xticks(np.arange(0, 1.2, .2))
ax.set_xticklabels(np.round(np.arange(0, 1.2, .2), 2), fontsize=20)
ax.set_yticks(np.arange(0, 1.2, .2))
ax.set_yticklabels(np.round(np.arange(0, 1.2, .2), 2), fontsize=20)
ax.set_ylabel('Observed', fontsize=25)
ax.grid(False)
plt.savefig(config.FIGURES_DIR / 'fraction_cres_in_coding_regions.pdf', bbox_inches='tight', format='pdf')
plt.close()

### Distances from CREs to GenBank gene start positions

Fraction of viral CREs proximal to GenBank gene start positions: observed vs. expected fractions

In [6]:
cres_tss_fractions_df = pd.read_csv(config.RESULTS_DIR / "cres_proximal_to_gene_starts_fractions.csv")

fig, ax = plt.subplots(figsize=(10, 10), layout="tight")
ax.set_title('Fraction of viral CREs proximal to GenBank gene starts', fontsize=25)
for family, fraction_df in cres_tss_fractions_df.groupby('family', sort=True, observed=True):
    ax.scatter(fraction_df.expected, fraction_df.observed, alpha=.7, s=250, color=config.DSDNA_FAMILY_COLORS[family], label=family)  # add outline: edgecolor='black'
ax.plot([0, 1], [0, 1], ls='-', color='black')
ax.legend(fontsize=20, loc='upper left')
ax.set_xlabel('Expected', fontsize=25)
ax.set_xticks(np.arange(0, 1.2, .2))
ax.set_xticklabels(np.round(np.arange(0, 1.2, .2), 2), fontsize=20)
ax.set_yticks(np.arange(0, 1.2, .2))
ax.set_yticklabels(np.round(np.arange(0, 1.2, .2), 2), fontsize=20)
ax.set_ylabel('Observed', fontsize=25)
ax.grid(False)
plt.savefig(config.FIGURES_DIR / 'fraction_cres_proximal_to_gene_starts.pdf', bbox_inches='tight', format='pdf')
plt.close()

**Fraction of GenBank gene start positions proximal to CREs: observed vs. expected**

The approximate expected value is calculated as  
$T \cdot \left[ 1 - \left(1 - \frac{2 R}{G}\right)^C \right]$,  
where:
- $T$ is the number of transcription start sites (TSSs),
- $R$ is the TSS context size (`PROXIMITY_RANGE`, e.g., 250 bp),
- $G$ is the genome size, and
- $C$ is the number of CREs.

This formula is an approximation based on the assumption that TSS and CRE positions are independently and uniformly distributed across the genome.


In [7]:
CELL = 'HEK293'

tss_cres_fractions_df = pd.read_csv(config.RESULTS_DIR / f"gene_starts_proximal_to_cres_fractions_{CELL.lower()}.csv")

fig, ax = plt.subplots(figsize=(10, 10), layout="tight")
ax.set_title('Fraction of GenBank gene starts proximal to viral CREs', fontsize=25)
for family, fraction_df in tss_cres_fractions_df.groupby('family', sort=True, observed=True):
    ax.scatter(fraction_df.expected, fraction_df.observed, alpha=.7, s=250, color=config.DSDNA_FAMILY_COLORS[family], label=family)  # add outline: edgecolor='black'
ax.plot([0, 1], [0, 1], ls='-', color='black')
ax.legend(fontsize=20, loc='upper right')
ax.set_xlabel('Expected', fontsize=25)
ax.set_xticks(np.arange(0, 1.2, .2))
ax.set_xticklabels(np.round(np.arange(0, 1.2, .2), 2), fontsize=20)
ax.set_yticks(np.arange(0, 1.2, .2))
ax.set_yticklabels(np.round(np.arange(0, 1.2, .2), 2), fontsize=20)
ax.set_ylabel('Observed', fontsize=25)
ax.grid(False)
plt.savefig(config.FIGURES_DIR / 'fraction_gene_starts_proximal_to_cres.pdf', bbox_inches='tight', format='pdf')
plt.close()

### Comparison with CAGE-seq Datasets

The following datasets were used for comparison (processed in the [`process_cage_datasets.py`](../scripts/process_cage_datasets.py) script):

| Virus (Name)                               | Experimental Methods                      | Cell Line(s)                         | PMID        | Viral Genome Accession (GenBank ID) | Source             |
|-------------------------------------------|-------------------------------------------|--------------------------------------|-------------|--------------------------------------|---------------------|
| HHV-1 (Herpes simplex virus 1)            | RNA-seq, cRNA-seq, dRNA-seq               | HFF                                   | 32341360    | BK012101.1                           | GenBank record      |
| HHV-3 (Varicella-zoster virus)            | CAGE-seq, RNA-seq                         | ARPE-19                               | 33024035    | NC_001348.1                          | Table S1            |
| HHV-4 (Epstein–Barr virus)                | CAGE-seq                                  | HEK293                                | 29864140    | V01555.2                             | Table S1            |
| HHV-8 (Kaposi’s sarcoma-associated herpesvirus) | CAGE-seq, long-read sequencing (LRS) | PEL, hESC-derived neurons             | 38206015    | GQ994935.1                           | Supplemental Tables |


In [8]:
paired_tiles_tss = pd.read_csv(config.RESULTS_DIR / "virus_paired_tiles_cage_peaks_overlap.csv")

for virus, genome in [
    ['HHV-1', 'BK012101.1'], ['HHV-3', 'NC_001348.1'], ['HHV-4', 'V01555.2'], ['HHV-8', 'GQ994935.1']
]:
    df = paired_tiles_tss[paired_tiles_tss['genome'] == genome]
    fig, axes = plt.subplots(2, 3, figsize=(15, 10), sharey=True, layout="tight")
    axes = axes.flatten()
    POS_STEP = 7e1
    for i, cell in enumerate(config.CELL_LINES):
        ax = axes[i]
        cell_df = df[df['cell'] == cell]
        tss_ranks = cell_df[cell_df['is_cage_peak']]['cell_rank'].values.astype(np.float64)
        not_tss_ranks = cell_df[~cell_df['is_cage_peak']]['cell_rank'].values.astype(np.float64)
        plots.violin(ax, tss_ranks, pos=-POS_STEP, box_width=25, violincolor='red')
        plots.violin(ax, not_tss_ranks, pos=POS_STEP, box_width=25, violincolor='red')
        stat, p = sps.mannwhitneyu(tss_ranks, not_tss_ranks, alternative='two-sided')
        ax.set_title(f'{cell} ({p:.4e})')
        ax.set_xticks([-POS_STEP, POS_STEP])
        ax.set_xticklabels(['TSS', 'not TSS'])
        ax.set_xlabel('')
        if i % 3 == 0:
            ax.set_ylabel('Cell activity rank')
        ax.grid(False)
    fig.suptitle(f'Five-prime dataset for {virus} (Mann-Whitney U p-values)', fontsize=16)
    plt.savefig(
        config.FIGURES_DIR / f"tiles_cage_overlap_rank_activity_distribution_{virus}.pdf",
        format="pdf", bbox_inches="tight"
    )
    plt.close()


Stratified by both CDS and CAGE-peak overlap pattern

In [9]:
paired_tiles_tss = pd.read_csv(config.RESULTS_DIR / "virus_paired_tiles_cage_peaks_overlap.csv")

for virus, genome in [
    ['HHV-1', 'BK012101.1'], ['HHV-3', 'NC_001348.1'], ['HHV-4', 'V01555.2'], ['HHV-8', 'GQ994935.1']
]:
    df = paired_tiles_tss[paired_tiles_tss['genome'] == genome]
    fig, axes = plt.subplots(2, 3, figsize=(18, 10), sharey=True, layout="tight")
    axes = axes.flatten()
    POS_STEP = 1.3e2
    for i, cell in enumerate(config.CELL_LINES):
        ax = axes[i]
        cell_df = df[df['cell'] == cell]
        conditions = [
            (True, False),   # TSS, not CDS
            (False, False),  # not TSS, not CDS
            (True, True),    # TSS, CDS
            (False, True),   # not TSS, CDS
        ]
        xticks = []
        xticklabels = []
        all_ranks = []
        for j, (is_tss, is_cds) in enumerate(conditions):
            subset = cell_df[(cell_df['is_cage_peak'] == is_tss) & (cell_df['is_cds'] == is_cds)]
            ranks = subset['cell_rank'].values.astype(np.float64)
            plots.violin(ax, ranks, pos=(j - 1.5) * POS_STEP, bw_method=.2, box_width=25, violincolor='red')
            all_ranks.append(ranks)
            xticks.append((j - 1.5) * POS_STEP)
            xticklabels.append(f"{'TSS' if is_tss else 'not TSS'}\n{'CDS' if is_cds else 'not CDS'}")
        # statistical test: TSS vs not TSS, across all CDS statuses
        tss_ranks = cell_df[cell_df['is_cage_peak'] == True]['cell_rank'].values.astype(np.float64)
        not_tss_ranks = cell_df[cell_df['is_cage_peak'] == False]['cell_rank'].values.astype(np.float64)
        stat, p = sps.mannwhitneyu(tss_ranks, not_tss_ranks, alternative='two-sided')
        ax.set_title(f'{cell} ({p:.2e})')
        ax.set_xticks(xticks)
        ax.set_xticklabels(xticklabels)
        ax.set_xlabel('')
        if i % 3 == 0:
            ax.set_ylabel('Cell activity rank')
        ax.grid(False)

    fig.suptitle(f'Five-prime dataset for {virus} (Mann-Whitney U p-values)', fontsize=16)
    plt.savefig(
        config.FIGURES_DIR / f"tiles_cage_cds_overlap_rank_activity_distribution_{virus}.pdf",
        format="pdf", bbox_inches="tight"
    )
    plt.close()


### MPRA Activity Across Herpesvirus Kinetic Groups

Gene kinetic classifications were primarily based on *Fields Virology* (Knipe & Howley, 6th ed., 2013). When annotations were missing or outdated, we supplemented them with peer-reviewed sources listed in the supplementary table.


In [10]:
HERPESVIRUSES = [nm.removeprefix('Herpesviridae, ') for nm in config.VIRUSES if nm.startswith('Herpesviridae')]
herpes_kinetics_df = pd.read_csv(config.RAW_DIR / 'herpesvirus_tss_kinetics_manual_annotation.csv')
herpes_kinetics_df.sample(2, random_state=0)


,strain,genome,gene_name,strand,five_prime,IE,E,E/L,L,Latent
549,Kaposi Sarcoma (HHV-8),MZ712172.1,HHV8_gs01,+,28820,False,True,False,False,False
502,Kaposi Sarcoma (HHV-8),MZ712172.1,ORF29,-,54775,False,False,False,True,False


Find the maximum cell MPRA activity rank between all tiles whose midpoint resides within the $\pm 250$ promoter proximity region. For each kinetic group plot the average of these values across all promoters.

In [11]:
herpes_kinetics_stats_df = herpes_kinetics_df.drop(['genome', 'gene_name', 'strand', 'five_prime'], axis=1).groupby('strain').sum(0).reset_index()
herpes_kinetics_activity_df = pd.read_csv(config.RESULTS_DIR / 'herpesvirus_tss_kinetics_average_cell_rank.csv')

fig, axes = plt.subplots(figsize=(28, 14), nrows=2, ncols=4)
for strain_idx, strain in enumerate(HERPESVIRUSES):
    ax = axes[strain_idx // 4, strain_idx % 4]
    stats = herpes_kinetics_stats_df[herpes_kinetics_stats_df.strain == strain][config.GENE_KINETIC_GROUPS].values
    stats = np.tile(stats, len(config.CELL_LINES)).reshape(len(config.CELL_LINES), -1)
    ranks_df = herpes_kinetics_activity_df[herpes_kinetics_activity_df.strain == strain].drop('strain', axis=1)
    img = plots.heatmap_with_stats(
        ax, ranks_df.set_index('cell'), imshow_args={'cmap': 'RdBu_r', 'vmin': .7, 'vmax': 1, 'norm': None},
        title_args={'label': strain, 'fontsize': 20}, text_values=stats,
    )
    if strain_idx == 3:
        cbar_ax = fig.add_axes([.92, 0.15, 0.02, 0.7])
        cbar = fig.colorbar(img, cax=cbar_ax)
        cbar.set_label("Tile cell activity rank")
    ax.set_xticklabels(config.GENE_KINETIC_GROUPS, fontsize=15)
plt.subplots_adjust(wspace=0.25)
fig.savefig(config.FIGURES_DIR / 'herpesvirus_tss_kinetics_stats.pdf', bbox_inches="tight", format="pdf")
plt.close()


**Gene Kinetic Classification of Epstein–Barr Virus (HHV-4)**

This analysis is based on the gene expression kinetics reported by Reza Djavadian *et al.* (2018), PMID: [29864140](https://pubmed.ncbi.nlm.nih.gov/29864140). The study categorizes Epstein–Barr Virus (EBV, Human herpesvirus 4) genes into kinetic classes based on temporal expression profiles during the lytic cycle.


In [12]:
HHV4_KINETICS = ['early', 'leaky', 'late', 'latent']
hhv4_kinetics = pd.read_csv(config.RESULTS_DIR / 'hhv4_cage_pmid_29864140_kinetics_cell_ranks.csv')
hhv4_kinetics.kinetics = hhv4_kinetics.kinetics.astype('category').cat.set_categories(HHV4_KINETICS)
hhv4_kinetics.cell = hhv4_kinetics.cell.astype('category').cat.set_categories(config.CELL_LINES)

hhv4_kinetics_stats = hhv4_kinetics[['cell', 'kinetics', 'max_cell_rank']].groupby(['cell', 'kinetics'], observed=False).size().reset_index().rename({0: 'counts'}, axis=1)
hhv4_kinetics_stats = hhv4_kinetics_stats.pivot(columns='kinetics', index='cell', values='counts').reset_index(drop=True).values
hhv4_kinetics_activity = hhv4_kinetics[['cell', 'kinetics', 'max_cell_rank']].groupby(['cell', 'kinetics'], observed=False).mean('max_cell_rank').reset_index()
hhv4_kinetics_activity = hhv4_kinetics_activity.pivot(columns='kinetics', index='cell', values='max_cell_rank').reset_index().set_index('cell')

fig, ax = plt.subplots(figsize=(4, 4))
img = plots.heatmap_with_stats(
    ax, hhv4_kinetics_activity, imshow_args={'cmap': 'RdBu_r', 'vmin': .7, 'vmax': 1, 'norm': None},
    title_args={'label': 'Epstein Barr Virus (HHV-4)', 'fontsize': 20}, text_values=hhv4_kinetics_stats,
)
cbar_ax = fig.add_axes([.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(img, cax=cbar_ax)
cbar.set_label("Tile cell activity rank")
ax.set_xticklabels(HHV4_KINETICS, fontsize=15)
fig.savefig(config.FIGURES_DIR / 'hhv4_cage_pmid_29864140_kinetics_stats.pdf', bbox_inches="tight", format="pdf")
plt.close()
